In [57]:
'''
   "51":  "Right lateral ventricle",
   "52":  "Left lateral ventricle",
   "4":  "third ventricle",
   "11":  "fourth ventricle"
   
   "6": acquedukt
'''

'\n   "51":  "Right lateral ventricle",\n   "52":  "Left lateral ventricle",\n   "4":  "third ventricle",\n   "11":  "fourth ventricle"\n   \n   "6": acquedukt\n'

In [2]:
import glob
import nibabel as nib
import numpy as np
import os
import cupy as cp
import pandas as pd
import re
import time
import seaborn as sns
from joblib import Parallel, delayed

In [59]:
def dice(f1, f2,n):
    arr1 = nib.load(f1).get_fdata()
    arr2 = nib.load(f2).get_fdata()
    arr1 = np.where(arr1==n,1,0)
    arr2 = np.where(arr2==n,1,0)
    return (arr1 & arr2).sum()*2/(arr1.sum() + arr2.sum())

In [3]:
manual_mask = '/home/orco/data/VentrikelCNN/derivatives/ventricle_seg_manual/sub-*/ses-01/anat/sub-V*_ses-01_space-T1w_desc-ventricle-mask.nii.gz'
gt = glob.glob(manual_mask)

In [4]:
#gt

In [5]:
subids = [i.split('/')[-4] for i in gt]


In [6]:
len(subids)

146

In [68]:
subids

['sub-V104',
 'sub-V102',
 'sub-V148',
 'sub-V023',
 'sub-V037',
 'sub-V011',
 'sub-V030',
 'sub-V145',
 'sub-V012',
 'sub-V120',
 'sub-V046',
 'sub-V073',
 'sub-V114',
 'sub-V142',
 'sub-V101',
 'sub-V072',
 'sub-V018',
 'sub-V124',
 'sub-V134',
 'sub-V150',
 'sub-V021',
 'sub-V122',
 'sub-V087',
 'sub-V080',
 'sub-V077',
 'sub-V041',
 'sub-V065',
 'sub-V050',
 'sub-V004',
 'sub-V001',
 'sub-V006',
 'sub-V105',
 'sub-V049',
 'sub-V036',
 'sub-V131',
 'sub-V005',
 'sub-V129',
 'sub-V112',
 'sub-V057',
 'sub-V109',
 'sub-V017',
 'sub-V009',
 'sub-V135',
 'sub-V066',
 'sub-V069',
 'sub-V014',
 'sub-V128',
 'sub-V068',
 'sub-V040',
 'sub-V088',
 'sub-V140',
 'sub-V058',
 'sub-V059',
 'sub-V071',
 'sub-V019',
 'sub-V078',
 'sub-V121',
 'sub-V133',
 'sub-V010',
 'sub-V031',
 'sub-V090',
 'sub-V002',
 'sub-V047',
 'sub-V144',
 'sub-V003',
 'sub-V075',
 'sub-V034',
 'sub-V061',
 'sub-V033',
 'sub-V024',
 'sub-V103',
 'sub-V126',
 'sub-V086',
 'sub-V035',
 'sub-V022',
 'sub-V147',
 'sub-V118',

In [13]:
cnn = '/home/orco/data/VentrikelCNN/derivatives/ventricle_seg/{subid}/{subid}_n4_mni_strip_seg_inverse.nii.gz'
gt = '/home/orco/data/VentrikelCNN/derivatives/ventricle_seg_manual/{subid}/ses-01/anat/{subid}_ses-01_space-T1w_desc-ventricle-mask.nii.gz'

In [14]:
roinums = [51, 52, 4, 11]

In [15]:
cnns =[cnn.format(subid=i) for i in subids]
gtss =[gt.format(subid=i) for i in subids]

In [16]:
cnns =[nib.load(i) for i in cnns]
gtss =[nib.load(i) for i in gtss]

FileNotFoundError: No such file or no access: '/home/orco/data/VentrikelCNN/derivatives/ventricle_seg/sub-V291/sub-V291_n4_mni_strip_seg_inverse.nii.gz'

In [75]:
start = time.time()
d = dict()
for i in subids:
    try:
        d[i] = Parallel(n_jobs=4)(delayed(dice)(cnn.format(i),gt.format(i),n) for n in roinums)
        print('made: ', i)
    except:
        print('skipped: ', i)
stop = time.time()
print(stop-start)
#748.1604852676392s

skipped:  sub-V104
skipped:  sub-V102
skipped:  sub-V148
skipped:  sub-V023
skipped:  sub-V037
skipped:  sub-V011
skipped:  sub-V030
skipped:  sub-V145
skipped:  sub-V012
skipped:  sub-V120
skipped:  sub-V046
skipped:  sub-V073
skipped:  sub-V114
skipped:  sub-V142
skipped:  sub-V101
skipped:  sub-V072
skipped:  sub-V018
skipped:  sub-V124
skipped:  sub-V134
skipped:  sub-V150
skipped:  sub-V021
skipped:  sub-V122
skipped:  sub-V087
skipped:  sub-V080
skipped:  sub-V077
skipped:  sub-V041
skipped:  sub-V065
skipped:  sub-V050
skipped:  sub-V004
skipped:  sub-V001
skipped:  sub-V006
skipped:  sub-V105
skipped:  sub-V049
skipped:  sub-V036
skipped:  sub-V131
skipped:  sub-V005
skipped:  sub-V129
skipped:  sub-V112
skipped:  sub-V057
skipped:  sub-V109
skipped:  sub-V017
skipped:  sub-V009
skipped:  sub-V135
skipped:  sub-V066
skipped:  sub-V069
skipped:  sub-V014
skipped:  sub-V128
skipped:  sub-V068
skipped:  sub-V040
skipped:  sub-V088
skipped:  sub-V140
skipped:  sub-V058
skipped:  su